# Real-Time Control to Sound Demo

This notebook demonstrates the core BrainJam performance loop:
- **Input**: Keyboard or mock EEG controls
- **Mapping**: Control parameters to synthesis parameters
- **Synthesis**: Real-time audio generation
- **Feedback**: Immediate audio output

**Target Latency**: < 100ms end-to-end

---

## Important Notes

- This is a **performance instrument**, not brain decoding
- Brain signals (when used) are **control inputs**, comparable to breath or gesture
- You maintain **full creative control** - this is not autonomous AI
- Start with mock EEG or keyboard to understand the system

In [ ]:
# Import required libraries
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display, clear_output
import time

# Import BrainJam performance system
from performance_system.controllers import MockEEGController
from performance_system.sound_engines import ParametricSynth
from performance_system.mapping_models import LinearMapper

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Initialize the Performance System

We'll set up:
- **Controller**: Mock EEG (generates structured control signals)
- **Synthesizer**: Parametric synth with 4 controllable parameters
- **Mapper**: Optional mapping layer (identity for now)

In [ ]:
# Initialize controller
controller = MockEEGController(fs=250)  # 250 Hz sampling rate
print("✓ Mock EEG Controller initialized")
print("  Generates 4 continuous control signals (0-1 range)")
print("  NOT decoding thoughts - just structured test signals")

# Initialize synthesizer
synth = ParametricSynth(sample_rate=44100, base_freq=220.0)  # A3
print("\n✓ Parametric Synthesizer initialized")
print("  Controllable parameters:")
print("    - tempo_density: Event rate (0=sparse, 1=dense)")
print("    - harmonic_tension: Dissonance (0=consonant, 1=tense)")
print("    - spectral_brightness: Timbre (0=dark, 1=bright)")
print("    - noise_balance: Texture (0=tonal, 1=noisy)")

# Optional: Initialize mapper (identity mapping for now)
mapper = LinearMapper(n_inputs=4, n_outputs=4)
print("\n✓ Linear Mapper initialized (identity mapping)")

## 2. Test the Control Signal

Let's see what kind of control signals the mock EEG generates.
These are continuous, slowly-varying parameters - NOT decoded mental states.

In [ ]:
# Generate a sequence of control vectors
n_samples = 100
control_history = []

for i in range(n_samples):
    controls = controller.get_control_vector(duration=0.1)
    control_history.append(controls)

# Convert to arrays for plotting
control_1 = [c['control_1'] for c in control_history]
control_2 = [c['control_2'] for c in control_history]
control_3 = [c['control_3'] for c in control_history]
control_4 = [c['control_4'] for c in control_history]

# Plot
fig, axes = plt.subplots(4, 1, figsize=(12, 8))
time_axis = np.arange(n_samples) * 0.1  # Time in seconds

axes[0].plot(time_axis, control_1, 'b-', linewidth=2)
axes[0].set_ylabel('Control 1\n(Tempo/Density)', fontsize=10)
axes[0].set_ylim([0, 1])
axes[0].grid(True, alpha=0.3)

axes[1].plot(time_axis, control_2, 'g-', linewidth=2)
axes[1].set_ylabel('Control 2\n(Harmony)', fontsize=10)
axes[1].set_ylim([0, 1])
axes[1].grid(True, alpha=0.3)

axes[2].plot(time_axis, control_3, 'r-', linewidth=2)
axes[2].set_ylabel('Control 3\n(Brightness)', fontsize=10)
axes[2].set_ylim([0, 1])
axes[2].grid(True, alpha=0.3)

axes[3].plot(time_axis, control_4, 'm-', linewidth=2)
axes[3].set_ylabel('Control 4\n(Noise)', fontsize=10)
axes[3].set_ylim([0, 1])
axes[3].set_xlabel('Time (seconds)', fontsize=11)
axes[3].grid(True, alpha=0.3)

plt.suptitle('Mock EEG Control Signals Over Time', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nNote: These are continuous, smoothly-varying control signals.")
print("They are NOT decoded thoughts or mental states - just control parameters.")

## 3. Generate Audio from Control Signals

Now let's hear what these control signals sound like when mapped to synthesis parameters.

In [ ]:
# Reset controller and synth
controller.reset()
synth.reset()

# Generate 5 seconds of audio
duration = 5.0  # seconds
chunk_size = 0.1  # 100ms chunks
n_chunks = int(duration / chunk_size)

audio_chunks = []
control_log = []

print("Generating audio...")
for i in range(n_chunks):
    # Get control vector
    controls = controller.get_control_vector(duration=chunk_size)
    control_log.append(controls)
    
    # Generate audio chunk
    audio_chunk = synth.generate(chunk_size, controls)
    audio_chunks.append(audio_chunk)
    
    if (i + 1) % 10 == 0:
        print(f"  Generated {(i + 1) * chunk_size:.1f}s / {duration}s")

# Concatenate audio
audio = np.concatenate(audio_chunks)

print(f"\n✓ Generated {duration}s of audio ({len(audio)} samples at {synth.sample_rate} Hz)")
print("\nPlay the audio below:")

# Display audio player
display(Audio(audio, rate=synth.sample_rate))

## 4. Visualize Audio and Control Parameters Together

Let's see how the control signals correspond to the audio output.

In [ ]:
# Extract control parameters
control_1 = [c['control_1'] for c in control_log]
control_2 = [c['control_2'] for c in control_log]

# Downsample audio for visualization
downsample_factor = 1000
audio_downsampled = audio[::downsample_factor]
time_audio = np.arange(len(audio_downsampled)) * downsample_factor / synth.sample_rate

# Time axis for controls
time_controls = np.arange(len(control_1)) * chunk_size

# Plot
fig, axes = plt.subplots(3, 1, figsize=(12, 8))

# Audio waveform
axes[0].plot(time_audio, audio_downsampled, 'k-', linewidth=0.5, alpha=0.7)
axes[0].set_ylabel('Audio\nAmplitude', fontsize=10)
axes[0].set_xlim([0, duration])
axes[0].grid(True, alpha=0.3)
axes[0].set_title('Generated Audio Waveform', fontsize=11)

# Control 1 (Tempo/Density)
axes[1].plot(time_controls, control_1, 'b-', linewidth=2)
axes[1].set_ylabel('Control 1\n(Tempo/Density)', fontsize=10)
axes[1].set_ylim([0, 1])
axes[1].set_xlim([0, duration])
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0.5, color='gray', linestyle='--', alpha=0.3)

# Control 2 (Harmonic Tension)
axes[2].plot(time_controls, control_2, 'g-', linewidth=2)
axes[2].set_ylabel('Control 2\n(Harmony)', fontsize=10)
axes[2].set_ylim([0, 1])
axes[2].set_xlim([0, duration])
axes[2].set_xlabel('Time (seconds)', fontsize=11)
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=0.5, color='gray', linestyle='--', alpha=0.3)

plt.suptitle('Audio Output vs Control Parameters', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nObserve how changes in control parameters affect the audio:")
print("- Higher Control 1 → More frequent events (denser rhythm)")
print("- Higher Control 2 → More dissonant harmonics (tension)")

## 5. Interactive Control (Simple Version)

Let's try manually setting control values to hear the effect.
This demonstrates that **you are in control**, not some autonomous AI.

In [ ]:
# Function to generate audio with specific control values
def generate_with_controls(c1, c2, c3, c4, duration=2.0):
    """
    Generate audio with specific control parameter values.
    
    Args:
        c1: tempo_density (0-1)
        c2: harmonic_tension (0-1)
        c3: spectral_brightness (0-1)
        c4: noise_balance (0-1)
        duration: Length in seconds
    """
    controls = {
        'control_1': c1,
        'control_2': c2,
        'control_3': c3,
        'control_4': c4,
    }
    
    synth.reset()
    
    # Generate audio in chunks
    chunk_size = 0.1
    n_chunks = int(duration / chunk_size)
    audio_chunks = []
    
    for _ in range(n_chunks):
        audio_chunk = synth.generate(chunk_size, controls)
        audio_chunks.append(audio_chunk)
    
    audio = np.concatenate(audio_chunks)
    return audio

print("Try different control combinations:\n")

# Example 1: Sparse, consonant, bright, tonal
print("1. Sparse, consonant, bright, tonal")
audio1 = generate_with_controls(0.2, 0.2, 0.8, 0.1)
display(Audio(audio1, rate=synth.sample_rate))

# Example 2: Dense, tense, dark, noisy
print("\n2. Dense, tense, dark, noisy")
audio2 = generate_with_controls(0.8, 0.8, 0.3, 0.7)
display(Audio(audio2, rate=synth.sample_rate))

# Example 3: Medium everything
print("\n3. Medium settings (balanced)")
audio3 = generate_with_controls(0.5, 0.5, 0.5, 0.5)
display(Audio(audio3, rate=synth.sample_rate))

print("\n✓ Notice how you have direct control over the sound.")
print("  This is an instrument, not an autonomous system.")

## 6. Performance Metrics

Let's measure the system's responsiveness - a key factor for live performance.

In [ ]:
# Measure latency for different stages
n_trials = 100
latencies = {
    'control_generation': [],
    'audio_generation': [],
    'total': []
}

print("Measuring system latency...")

for i in range(n_trials):
    t_start = time.time()
    
    # Control generation
    t0 = time.time()
    controls = controller.get_control_vector(duration=0.1)
    t1 = time.time()
    latencies['control_generation'].append((t1 - t0) * 1000)  # ms
    
    # Audio generation
    t2 = time.time()
    audio = synth.generate(0.1, controls)
    t3 = time.time()
    latencies['audio_generation'].append((t3 - t2) * 1000)  # ms
    
    t_end = time.time()
    latencies['total'].append((t_end - t_start) * 1000)  # ms

# Calculate statistics
print("\n" + "="*60)
print("LATENCY MEASUREMENTS (100 trials)")
print("="*60)

for stage, times in latencies.items():
    mean_time = np.mean(times)
    std_time = np.std(times)
    min_time = np.min(times)
    max_time = np.max(times)
    
    print(f"\n{stage.replace('_', ' ').title()}:")
    print(f"  Mean: {mean_time:.2f} ms (± {std_time:.2f})")
    print(f"  Range: {min_time:.2f} - {max_time:.2f} ms")

total_mean = np.mean(latencies['total'])

print("\n" + "="*60)
if total_mean < 100:
    print(f"✓ PASS: Total latency ({total_mean:.1f} ms) < 100 ms target")
    print("  System is suitable for real-time performance!")
else:
    print(f"⚠ WARNING: Total latency ({total_mean:.1f} ms) > 100 ms target")
    print("  May need optimization for real-time use.")
print("="*60)

# Plot latency distribution
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].hist(latencies['control_generation'], bins=20, color='blue', alpha=0.7)
axes[0].set_xlabel('Latency (ms)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Control Generation')
axes[0].grid(True, alpha=0.3)

axes[1].hist(latencies['audio_generation'], bins=20, color='green', alpha=0.7)
axes[1].set_xlabel('Latency (ms)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Audio Generation')
axes[1].grid(True, alpha=0.3)

axes[2].hist(latencies['total'], bins=20, color='red', alpha=0.7)
axes[2].axvline(x=100, color='black', linestyle='--', linewidth=2, label='100ms target')
axes[2].set_xlabel('Latency (ms)')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Total End-to-End')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.suptitle('Latency Distribution (100 trials)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:

1. ✓ **Real-time control** — Generate control signals from mock EEG
2. ✓ **Audio synthesis** — Map controls to sound parameters
3. ✓ **Visualization** — See control-sound relationships
4. ✓ **Manual control** — Direct parameter manipulation
5. ✓ **Performance metrics** — Measure system latency

### Key Takeaways

- **This is a performance instrument**, not brain decoding
- **You are in control** — parameters respond to your input
- **Low latency** — System is responsive enough for live use
- **Transparent** — All mappings are visible and adjustable

### Next Steps

- Try the **AI co-performer demo** (`ai_co_performer_demo.ipynb`)
- Experiment with **custom mappings** (LinearMapper, MLPMapper)
- Connect **real EEG/fNIRS** hardware (if available)
- Use **keyboard control** for comparison with EEG
- **Perform live** with the system!

---

**Remember**: BrainJam is about **performer agency and creative expression**,  
not signal accuracy or autonomous AI.